In [1]:
import numpy as np
import pandas as pd
import math, re
import pickle

In [2]:
#dataset
PATH_X_TRAIN='./wili-2018/x_train.txt'
PATH_Y_TRAIN='./wili-2018/y_train.txt'
PATH_X_TEST='./wili-2018/x_test.txt'
PATH_Y_TEST='./wili-2018/y_test.txt'

#parameters
COUNT_NGRAMS=20
COUNT_SYMBOLS=3

<h1>Preprocessing

In [3]:
#read dataset
def read_txt(path):
    handle = open(path, "r", encoding='utf-8')
    df = pd.DataFrame(handle.readlines())
    handle.close()
    return df

x_train=read_txt(PATH_X_TRAIN)
y_train=read_txt(PATH_Y_TRAIN)
x_test=read_txt(PATH_X_TEST)
y_test=read_txt(PATH_Y_TEST)

In [4]:
#clean text
def clean_text(text):
    new_text=''
    pos1=text.find('(')
    pos2=text.find(')')
    if pos2>pos1:
        text=text.replace(text[pos1:pos2+1],'')
    for t in text:
        if t.isalpha():
            new_text+=t
        elif t.isspace():
            new_text+=' '
    new_text=re.sub(" +"," ", new_text)
    return new_text.replace('\n','').lower().strip()

texts_train=[clean_text(s[0]) for s in x_train.values]
texts_test=[clean_text(s[0]) for s in x_test.values]

In [5]:
#classes preprocessing
classes_train=y_train.values
classes_test=y_test.values

classes = np.unique(np.array(classes_train))
nums=np.arange(len(classes))
d = dict(zip(classes,nums))

classes_train=[d[c[0]] for c in classes_train] 
classes_test=[d[c[0]] for c in classes_test] 

<h1>N-grams

In [6]:
#make trigrams for classes
def make_ngrams(text):
    ngrams=[]
    if len(text)>COUNT_SYMBOLS+1:
        for i in range(len(text)-COUNT_SYMBOLS+1):
            ngrams.append(text[i:i+COUNT_SYMBOLS])
    return ngrams

def add_ngrams(ngrams, l):
    for l1 in l:
        ngrams.add(l1[0])
    return ngrams

def collect_ngrams(texts, classes):#collect ngrams for all languages
    total_ngrams=set()
    for n in nums:
        ng_class=set()
        i=0
        for i in range(len(texts)):
            if classes[i]==n:
                ng_class.update(make_ngrams(texts[i]))
        ng_class=list(ng_class)
        ng_count=np.zeros(len(ng_class))
        for i in range(len(texts)):
            if classes[i]==n:
                ngrams=make_ngrams(texts[i])
                for ng in ngrams:
                    ng_count[ng_class.index(ng)]+=1
        ng_class = dict(zip(ng_class,ng_count))
        ng_class = sorted(ng_class.items(), key= lambda kv: kv[1], reverse=True)[:COUNT_NGRAMS] 
        total_ngrams = add_ngrams(total_ngrams, ng_class)
    return total_ngrams

def ng_freq(text):
    text=make_ngrams(text)
    freq=np.zeros(len(list_of_ngrams))
    for t in text:
        if t in list_of_ngrams:
            freq[list_of_ngrams.index(t)]+=1
    return freq

def ngrams_freq(texts, name):
    freq=[]
    for i, text in enumerate(texts):
        freq.append(ng_freq(text))
        if i%5000==0 or i==len(texts)-1:
            df=pd.DataFrame(freq)
            if i==0:
                df.to_csv(f, sep=';', index=False) 
            else:
                with open(name, 'a') as f:
                    df.to_csv(f, sep=';', index=False, header=False)
            freq.clear()

In [7]:
#set_of_ngrams = collect_ngrams(texts_train, classes_train)#prepare ngrams

with open('set_of_ngrams.dat', 'rb') as f:#load prepared ngrams
    set_of_ngrams=pickle.load(f)

In [8]:
#with open('set_of_ngrams.dat', 'wb') as f:#save ngrams
    #pickle.dump(set_of_ngrams, f)

In [9]:
list_of_ngrams=list(set_of_ngrams)
ngrams_freq(texts_train, 'ng_freq_train.csv')

In [10]:
ngrams_freq(texts_test, 'ng_freq_test.csv')